In [12]:
# Install the necessary libraries
!pip install -U -q diffusers accelerate transformers peft bitsandbytes
!pip install -q wandb # For metric identification and tracking

# Download the official training script for SDXL/SSD-1B
!wget https://raw.githubusercontent.com/huggingface/diffusers/main/examples/text_to_image/train_text_to_image_lora_sdxl.py

^C
ERROR: Operation cancelled by user
^C
ERROR: Operation cancelled by user
--2026-01-31 08:14:09--  https://raw.githubusercontent.com/huggingface/diffusers/main/examples/text_to_image/train_text_to_image_lora_sdxl.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56285 (55K) [text/plain]
Saving to: ‘train_text_to_image_lora_sdxl.py.2’

train_text_to_image 100%[===================>]  54.97K  --.-KB/s    in 0.01s   

2026-01-31 08:14:09 (4.63 MB/s) - ‘train_text_to_image_lora_sdxl.py.2’ saved [56285/56285]



In [ ]:
# 1. Uninstall existing version to avoid conflicts
!pip uninstall -y diffusers

# 2. Install the latest "dev" version from source
!pip install git+https://github.com/huggingface/diffusers

# 3. Ensure other dependencies are up to date for SSD-1B
!pip install -U -q accelerate transformers peft bitsandbytes

In [ ]:
import pandas as pd
import os
import shutil
import json
from tqdm import tqdm

# --- CONFIGURATION ---
# 1. PASTE YOUR COPIED CSV PATH HERE:
CSV_PATH = "/kaggle/input/labels/demo.csv" 

IMG_SOURCE_DIR = "/kaggle/input/pixel-art/images/images"
TRAIN_DIR = "/kaggle/working/train_data"
LIMIT = 3000 # Your project requirement
MANDATORY_KEYWORDS = "pixel art, 16-bit, sprite, "

os.makedirs(TRAIN_DIR, exist_ok=True)

# Load the CSV
df = pd.read_csv(CSV_PATH)
df_sample = df.sample(n=min(LIMIT, len(df)), random_state=42)

metadata = []
print(f"Moving {len(df_sample)} images to training folder...")

for index, row in tqdm(df_sample.iterrows(), total=len(df_sample)):
    file_name = row['file_name']
    caption = row['text']
    
    source_path = os.path.join(IMG_SOURCE_DIR, file_name)
    target_path = os.path.join(TRAIN_DIR, file_name)
    
    if os.path.exists(source_path):
        shutil.copy(source_path, target_path)
        # Combine your mandatory style keywords with the CSV caption
        # full_caption = MANDATORY_KEYWORDS + str(caption)
        full_caption = str(caption)
        metadata.append({"file_name": file_name, "text": full_caption})

# Save the metadata file that the training script expects
with open(os.path.join(TRAIN_DIR, "metadata.jsonl"), 'w') as f:
    for entry in metadata:
        f.write(json.dumps(entry) + "\n")

print(f"Successfully prepared {len(metadata)} images in {TRAIN_DIR}")

In [ ]:
!accelerate launch --num_processes=1 train_text_to_image_lora_sdxl.py \
  --pretrained_model_name_or_path="segmind/SSD-1B" \
  --pretrained_vae_model_name_or_path="madebyollin/sdxl-vae-fp16-fix" \
  --train_data_dir="/kaggle/working/train_data" \
  --caption_column="text" \
  --resolution=512 \
  --mixed_precision="fp16" \
  --train_batch_size=1 \
  --gradient_accumulation_steps=8 \
  --learning_rate=5e-5 \
  --rank=16 \
  --max_train_steps=2000 \
  --checkpointing_steps=500 \
  --validation_prompt="16x16 pixel art of a small owl flying" \
  --validation_epochs=1 \
  --num_validation_images=4 \
  --output_dir="/kaggle/working/court_of_owls_lora" \
  --seed=42 \
  --report_to="tensorboard" \
  --dataloader_num_workers=2

In [ ]:
# import matplotlib.pyplot as plt
# from PIL import Image
# import glob
# import os

# # Check multiple possible locations for validation images
# possible_paths = [
#     '/kaggle/working/court_of_owls_lora/samples/*.png',
#     '/kaggle/working/court_of_owls_lora/*.png',
#     '/kaggle/working/court_of_owls_lora/**/*.png'
# ]

# sample_images = []
# for path in possible_paths:
#     sample_images.extend(glob.glob(path, recursive=True))

# sample_images = list(set(sample_images))  # Remove duplicates
# sample_images.sort(key=os.path.getmtime)

# if sample_images:
#     # Show the last 4 validation images
#     num_to_show = min(4, len(sample_images))
#     fig, axes = plt.subplots(1, num_to_show, figsize=(5*num_to_show, 5))
#     if num_to_show == 1:
#         axes = [axes]
    
#     for ax, img_path in zip(axes, sample_images[-num_to_show:]):
#         img = Image.open(img_path)
#         ax.imshow(img)
#         ax.axis('off')
#         ax.set_title(os.path.basename(img_path))
    
#     plt.tight_layout()
#     plt.show()
# else:
#     print("No validation images found yet.")
#     print("Check output directory contents:")
#     !ls -la /kaggle/working/court_of_owls_lora/

No validation images generated yet. Wait for step 500!
